In [77]:
from openai import OpenAI
from getpass import getpass
import requests

from langchain.document_loaders import PyMuPDFLoader
from langchain.retrievers import ArxivRetriever

In [2]:
oai_key = getpass('Enter the key')

Enter the key········


In [3]:
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key= oai_key,
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What do you know about OpenAI and elon musk do they have some relation?",
        }
    ],
    model="gpt-3.5-turbo",
)

# Load Data

In [5]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("Resources/sec-guide-to-savings-and-investing.pdf")
pages = loader.load_and_split()

In [7]:
len(pages)

31

In [8]:
pages

[Document(page_content='A ROADMAP TO YOUR JOURNEY TO FINANCIAL SECURITY  |  1Saving and Investing\nA Roadmap To Your Financial Security \nThrough Saving and Investing', metadata={'source': 'Resources/sec-guide-to-savings-and-investing.pdf', 'page': 0}),
 Document(page_content='2  |  SAVING AND INVESTING', metadata={'source': 'Resources/sec-guide-to-savings-and-investing.pdf', 'page': 1}),
 Document(page_content='A ROADMAP TO YOUR JOURNEY TO FINANCIAL SECURITY  |  1\nDear Reader\nWhile money doesn’t grow on trees, it can grow when \nyou save and invest wisely.\nKnowing how to secure your financial well-being is one \nof the most important things you’ll ever need in life. Y ou don’t have to be a genius to do it. Y ou just need to know a few basics, form a plan, and be ready to stick to it. No matter how much or little money you have, the important thing is to educate yourself about your opportunities. In this brochure, we’ll cover the basics on saving and investing.\nAt the SEC, we enfor

## HTML

In [10]:
from langchain.document_loaders import UnstructuredURLLoader

urls = [
    "https://hbr.org/2021/08/how-to-make-smart-investments-a-beginners-guide",
    "https://www.nerdwallet.com/article/investing/how-to-invest-money",
    "https://www.nerdwallet.com/article/investing/investment-strategies"

]

loader = UnstructuredURLLoader(urls=urls)

data = loader.load()

In [14]:
data[1]

Document(page_content="MORE LIKE THIS\n\nInvestingInvesting Planning & Strategy\n\nInvesting money in the stock market is one of the main ways to build wealth and save for long-term goals such as retirement. But figuring out the best strategy to invest that money can feel daunting. That doesn't need to be the case, though —\xa0there are several straightforward, beginner-friendly ways to invest.\n\nAdvertisement\n\nCharles Schwab Interactive Brokers IBKR Lite Webull NerdWallet rating\xa0 NerdWallet's ratings are determined by our editorial team. The scoring formula for online brokers and robo-advisors takes into account over 15 factors, including account fees and minimums, investment choices, customer support and mobile app capabilities. 4.9 /5 NerdWallet rating\xa0 NerdWallet's ratings are determined by our editorial team. The scoring formula for online brokers and robo-advisors takes into account over 15 factors, including account fees and minimums, investment choices, customer suppor

In [13]:
len(data)

3

In [107]:
from concurrent.futures.thread import ThreadPoolExecutor
from langchain.document_loaders import PyMuPDFLoader
from langchain.retrievers import ArxivRetriever
from functools import partial
import requests
from bs4 import BeautifulSoup


class Scraper:
    """
    Scraper class to extract the content from the links
    """
    def __init__(self, urls):
        self.user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36 Edg/119.0.0.0"
        """
        Initialize the Scraper class.
        Args:
            urls:
        """
        self.urls = urls
        self.session = requests.Session()
        self.session.headers.update({
            "User-Agent": self.user_agent
        })

    def run(self):
        """
        Extracts the content from the links
        """
        partial_extract = partial(self.extract_data_from_link, session=self.session)
        with ThreadPoolExecutor(max_workers=20) as executor:
            contents = executor.map(partial_extract, self.urls)
        res = [content for content in contents if content['raw_content'] is not None]
        return res

    def extract_data_from_link(self, link, session):
        """
        Extracts the data from the link
        """
        content = ""
        try:
            if link.endswith(".pdf"):
                print('hhhhh', link)
                content = self.scrape_pdf_with_pymupdf(link)
            elif "arxiv.org" in link:
                doc_num = link.split("/")[-1]
                content = self.scrape_pdf_with_arxiv(doc_num)
            elif link:
                content = self.scrape_text_with_bs(link, session)

            if len(content) < 100:
                return {'url': link, 'raw_content': None}
            return {'url': link, 'raw_content': content}
        except Exception as e:
            return {'url': link, 'raw_content': None}

    def scrape_text_with_bs(self, link, session):
        response = session.get(link, timeout=4)
        soup = BeautifulSoup(response.content, 'lxml', from_encoding=response.encoding)

        for script_or_style in soup(["script", "style"]):
            script_or_style.extract()

        raw_content = self.get_content_from_url(soup)
        lines = (line.strip() for line in raw_content.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        content = "\n".join(chunk for chunk in chunks if chunk)
        return content

    def scrape_pdf_with_pymupdf(self, url) -> str:
        """Scrape a pdf with pymupdf

        Args:
            url (str): The url of the pdf to scrape

        Returns:
            str: The text scraped from the pdf
        """
        
        print('haha ', url)
        loader = PyMuPDFLoader(url)
        doc = loader.load()
        
        print('DOCCCC', doc)
        return str(doc)

    def scrape_pdf_with_arxiv(self, query) -> str:
        """Scrape a pdf with arxiv
        default document length of 70000 about ~15 pages or None for no limit

        Args:
            query (str): The query to search for

        Returns:
            str: The text scraped from the pdf
        """
        
        print('yo')
        retriever = ArxivRetriever(load_max_docs=2, doc_content_chars_max=None)
        docs = retriever.get_relevant_documents(query=query)
        return docs[0].page_content

    def get_content_from_url(self, soup):
        """Get the text from the soup

        Args:
            soup (BeautifulSoup): The soup to get the text from

        Returns:
            str: The text from the soup
        """
        text = ""
        tags = ['p', 'h1', 'h2', 'h3', 'h4', 'h5']
        for element in soup.find_all(tags):  # Find all the <p> elements
            text += element.text + "\n"
        return text

In [109]:
scraping = Scraper(['https://arxiv.org/pdf/2311.12755.pdf', ])
data = scraping.run()
print(data)


hhhhh https://arxiv.org/pdf/2311.12755.pdf
haha  https://arxiv.org/pdf/2311.12755.pdf
[]


In [100]:
len(data)

2

In [97]:
data[0]

{'url': 'https://hbr.org/2021/08/how-to-make-smart-investments-a-beginners-guide',
 'raw_content': 'Your Cart\nMaking Smart Investments: A Beginner’s Guide\nIf you make smart decisions and invest in the right places, you can reduce the risk factor, increase the reward factor, and generate meaningful returns.\xa0Here are a few questions to consider as you get started.\nWhere your work meets your life. See more from Ascend here.\nAre you a saver or spender?\nIf you went with the former, then you’re in the majority. According to a 2019 Charles Schwab survey, around 59% of Americans said they considered themselves\xa0savers. Compare that to more recent findings, however, and you’ll see\xa0that\xa063% of respondents\xa0in a similar demographic are currently living paycheck to paycheck.\nClearly, there’s a disconnect between the financial goals we are setting and the steps we are taking to realize them.\nMany of us are taught from a young age that saving is the most direct path to building w

In [113]:
loader = PyMuPDFLoader("https://arxiv.org/pdf/2310.06117.pdf?")
doc = loader.load()

print('DOCCCC', doc)


DOCCCC [Document(page_content='TAKE A STEP BACK: EVOKING REASONING VIA AB-\nSTRACTION IN LARGE LANGUAGE MODELS\nHuaixiu Steven Zheng∗\nSwaroop Mishra∗\nXinyun Chen\nHeng-Tze Cheng\nEd H. Chi\nQuoc V Le\nDenny Zhou\nGoogle DeepMind\nABSTRACT\nWe present STEP-BACK PROMPTING, a simple prompting technique that enables\nLLMs to do abstractions to derive high-level concepts and first principles from\ninstances containing specific details. Using the concepts and principles to guide the\nreasoning steps, LLMs significantly improve their abilities in following a correct\nreasoning path towards the solution. We conduct experiments of STEP-BACK\nPROMPTING with PaLM-2L models and observe substantial performance gains on\na wide range of challenging reasoning-intensive tasks including STEM, Knowl-\nedge QA, and Multi-Hop Reasoning. For instance, STEP-BACK PROMPTING\nimproves PaLM-2L performance on MMLU Physics and Chemistry by 7% and\n11%, TimeQA by 27%, and MuSiQue by 7%.\nThe purpose of abstracti

In [114]:
len(doc)

36

In [85]:
class Scrape:
    """Scraper class to scrape contents
    """
    def __init__(self, urls):
        self. urls = urls
        self.session = requests.Session()
        self.session.headers.update({
            "User-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36 Edg/119.0.0.0"
        })
        
        
    
    def scrape_text_with_bs(self, link, session) -> str:
        """Scraping text using beautifulSoup
        """
        
        response = session.get(link, timeout=4)
        soup = BeautifulSoup(response.content, 'lxml', from_encoding=response.encoding)

        
        for script_or_style in soup(["script", "style"]):
            script_or_style.extract() #remove all the styles and scripts
            
        
        raw_content = self.get_content_from_url(soup)
        
        lines = (line.strip() for line in raw_content.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        content = "\n".join(chunk for chunk in chunks if chunk)
        return content

        
        #print(raw_content)
    
    def get_content_from_url(self, soup):
        """Scrape the text from p,and h tags
        """
        
        text = ""
        tags = ['p', 'h1', 'h2', 'h3', 'h4', 'h5']
        for element in soup.find_all(tags):  # Find all the <p> elements
            
            print(f'{element.text} \n')
            text += element.text + "\n"
        return text
        
        
        
    def extract_data_from_link(self, link, session):
        """
        Extracts the data from the link
        """
        content = ""
        try:
            if link.endswith(".pdf"):
                content = self.scrape_pdf_with_pymupdf(link)
            elif "arxiv.org" in link:
                doc_num = link.split("/")[-1]
                content = self.scrape_pdf_with_arxiv(doc_num)
            elif link:
                content = self.scrape_text_with_bs(link, session)

            if len(content) < 100:
                return {'url': link, 'raw_content': None}
            return {'url': link, 'raw_content': content}
        except Exception as e:
            return {'url': link, 'raw_content': None}

    
    
    def scrape_pdf_with_pymupdf(self, url) -> str:
        """Scrape a pdf with pymupdf

        Args:
            url (str): The url of the pdf to scrape

        Returns:
            str: The text scraped from the pdf
        """
        loader = PyMuPDFLoader(url)
        doc = loader.load()
        return str(doc)

    def scrape_pdf_with_arxiv(self, query) -> str:
        """Scrape a pdf with arxiv
        default document length of 70000 about ~15 pages or None for no limit

        Args:
            query (str): The query to search for

        Returns:
            str: The text scraped from the pdf
        """
        retriever = ArxivRetriever(load_max_docs=2, doc_content_chars_max=None)
        docs = retriever.get_relevant_documents(query=query)
        return docs[0].page_content
    
    
        
    def run(self):
        """Extracts the content from the links
        """
        
        #partial function
        partial_extract = partial(self.extract_data_from_link, session = self.session)
        
        with ThreadPoolExecutor(max_workers=20) as executor:
            contents = executor.map(partial_extract, self.urls)
        res = [content for content in contents if content['raw_content'] is not None]
        return res

In [86]:
scraping = Scrape(['https://hbr.org/2021/08/how-to-make-smart-investments-a-beginners-guide', "https://www.nerdwallet.com/article/investing/how-to-invest-money", "https://www.nerdwallet.com/article/investing/investment-strategies"])

In [87]:
scraping.run()

 

Your Cart 

 


				Making Smart Investments: A Beginner’s Guide
			 

If you make smart decisions and invest in the right places, you can reduce the risk factor, increase the reward factor, and generate meaningful returns. Here are a few questions to consider as you get started. 

Where your work meets your life. See more from Ascend here. 

Are you a saver or spender? 

Partner Center 



[{'url': 'https://hbr.org/2021/08/how-to-make-smart-investments-a-beginners-guide',
  'raw_content': 'Your Cart\nMaking Smart Investments: A Beginner’s Guide\nIf you make smart decisions and invest in the right places, you can reduce the risk factor, increase the reward factor, and generate meaningful returns.\xa0Here are a few questions to consider as you get started.\nWhere your work meets your life. See more from Ascend here.\nAre you a saver or spender?\nPartner Center'}]